In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
# Load all three datasets:
data_prices = pd.read_csv('houses_price.csv')
data_address = pd.read_csv('houses_address.csv')
data_details = pd.read_csv('houses_details.csv')
data_population = pd.read_csv('houses_population.csv')
data_neighbourhood = pd.read_csv('houses_neighborhood.csv', encoding='latin-1')

In [28]:
######################################################################
###################INSERT FUNCTIONS HERE##############################
######################################################################

# NEIGHBOURHOOD
# extract distances from the points of interest:
import ast
import re
def extractDistances(x):
    if str(x) != 'nan':
        x = ast.literal_eval(x)[1]
        word_list = x.split(' ') 
        distance = word_list[0]
        units = word_list[1]
        distance = float(distance.replace(',', '.'))
        if units == 'kilometer':
            distance = distance*1000
    else: 
        distance = np.nan
    return distance

# DETAILS
# use this function on any column that has m, m^2, m^3 in it
# make for data_details dataframe.
def removeUnitsMeters(x):
    if str(x) not in ['-', 'nan', 'NaN']:
        if isinstance(x, float):
            x = x
        else:
            x = float(x.split(' ')[0])
    else:
        x = np.nan
    return x

# POPULATION
# apply to: data_population.distribution_fm = data_population.distribution_fm.apply(female_to_male_ratio)
def female_to_male_ratio(x):
    if (str(x) not in ['nan', 'NaN', '-']):
        info_both = re.findall("\d+\.\d+", x)
        if len(info_both)>1:
            males = float(info_both[1])
            females = float(info_both[0])
            ratio = females/males
        else:
            ratio = np.nan
    else:
        ratio = np.nan
    return ratio

def get_population_density(x):
    if (str(x) not in ['nan', 'NaN', '-']):
        info_both = re.findall("\d+\.\d+", x)
        if len(info_both)>0:
            density = float(info_both[0])
        else:
            density = np.nan
    else:
        density = np.nan
    return density

# returns percentages in 0-100!
def get_percentage(x):
    if (str(x) not in ['nan', 'NaN', '-']):
        info_both = re.findall("\d+\.\d+", x)
        if len(info_both)==0:
            info_both = re.findall("\d+", x)
            if len(info_both)>0:
                percentage = float(info_both[0])
                if len(info_both)>1:
                    percentage = percentage + float(info_both[1])/100
            else:
                percentage = np.nan
        elif len(info_both)>1:
            percentage = float(info_both[0]) + float(info_both[1])/100
    else:
        percentage = np.nan
    return percentage

def remove_currency(x):
    if (str(x) not in ['nan', 'NaN', '-']):
        info_both = re.findall("\d+\.\d+", x)
        if len(info_both)>0:
            euros = float(info_both[0])
        else:
            euros = np.nan
        return euros

# PRICES:

def fix_price(x):
    price_list = re.findall("\d+", x)
    if len(price_list)>0:
        if len(price_list)>2:
            price = float(price_list[0])*1e6 + float(price_list[1])*1e3 + float(price_list[2])
        elif len(price_list) == 2:
            price = float(price_list[0])*1e3 + float(price_list[1])
        elif len(price_list) == 1:
            price = float(price_list[0])*1e3
    else:
        price = 0
    return price

# remove letters from zip codes:
def remove_letters_zipcode(x):
    zipcode = re.findall("\d+", x)
    if len(zipcode)==1:
        to_return = int(zipcode[0])
    else:
        to_return = np.nan
    return to_return

In [29]:
# Use extractDistances(x) to get relevant information from the neighbourhood dataframe:
data_neighbourhood = data_neighbourhood.applymap(extractDistances)

# Use removeUnitsMeters to remove m2 and m3 from specific columns
data_details.Living_area = data_details.Living_area.apply(removeUnitsMeters)
data_details.LOT = data_details.LOT.apply(removeUnitsMeters)
data_details.Plot = data_details.Plot.apply(removeUnitsMeters)

# Fix female to male ratio:
data_population.distribution_fm = data_population.distribution_fm.apply(female_to_male_ratio)

# Fix population density:
data_population.population_density = data_population.population_density.apply(get_population_density)

# Fix population columns:
perc_cols = ['Age_0-15', 'age_15-25', 'age_25-45', 'age_45-65', 'age_older', 'indigenous', 'western_allochtoon', 'none_western', 'household_single', 'household_wo_kids', 'household_with_kids', 'perc_with_job', 'high_income', 'medium_income', 'low_income', 'social_benefit'] 
data_population.loc[:, perc_cols] = data_population.loc[:, perc_cols].applymap(get_percentage)
data_population.loc[:, 'income_avg'] = data_population.loc[:, 'income_avg'].apply(remove_currency)

# Fix property prices:
cols_to_fix = ['current_price', 'original_price', 'changes_in_price']
data_prices.loc[:, cols_to_fix] = data_prices.loc[:, cols_to_fix].applymap(fix_price)

# Fix zipcodes:
data_address.Zipcode = data_address.Zipcode.apply(remove_letters_zipcode)

In [30]:
# Concatenate all data to get a complete data frame:
data_frame = pd.concat([data_address, data_prices, data_details, data_neighbourhood, data_population], axis=1)

At this point, we got out dataset. But we do not neeed all of the variables that are listed in there. For this reason, I will remove some that we do not need in our model.

In [35]:
data_frame.columns

Index(['Street', 'Zipcode', 'Price', 'Broker', 'posted_date', 'current_price',
       'original_price', 'changes_in_price', 'price', 'price_per_m2',
       'times_in_sales', 'Type', 'Construction_year', 'Living_area', 'LOT',
       'Plot', 'Other', 'Insulation', 'Heating', 'Energy_label',
       'Energy_consumption', 'inside_maintenance_state', 'Rooms', 'Bedrooms',
       'Sanitation', 'kitchen', 'outside_maintenace_state',
       'outside_state_painting', 'Graden', 'view', 'Balcony', 'Garage',
       'number_of_times_shown', 'number_of_times_shown_yesterday',
       'treinstation', 'tankstation', 'supermarkt', 'basisschool',
       'kinderopvang', 'middelbare school', 'café', 'videotheek', '(huis)arts',
       'tandarts', 'fitnesscentrum', 'bibliotheek', 'inhabitants',
       'distribution_fm', 'population_density', 'Age_0-15', 'age_15-25',
       'age_25-45', 'age_45-65', 'age_older', 'indigenous',
       'western_allochtoon', 'none_western', 'household', 'household_single',
       '

In [36]:
data_frame.drop(columns = ['Street', 'Broker', 'posted_date', 'number_of_times_shown', 'number_of_times_shown_yesterday', 'videotheek'], inplace = True)

In [37]:
data_frame.head()

,Zipcode,Price,current_price,original_price,changes_in_price,price,price_per_m2,times_in_sales,Type,Construction_year,...,household_single,household_wo_kids,household_with_kids,avg_person_per_household,perc_with_job,high_income,medium_income,low_income,income_avg,social_benefit
0,1034.0,585.000,585000,585000,0,585.000,3.656,Nieuw,Herenhuis,1995,...,43.0,21.0,37.0,"2,2",63.0,11.0,46.0,43.0,16.4,10.07
1,1067.0,325.000,325000,325000,0,325.000,3.250,Nieuw,Hoekwoning,1955,...,43.0,20.0,37.0,"2,2",62.0,9.0,45.0,46.0,15.5,11.07
2,1086.0,910.000,910000,910000,0,910.000,3.889,5 dagen,Herenhuis,2008,...,30.0,18.0,53.0,"2,6",81.0,37.0,40.0,23.0,23.4,5.00
3,1016.0,1.050.000,1.05e+06,1.05e+06,0,1.050.000,7.554,Nieuw,Dubbele bovenwoning,1620,...,65.0,23.0,12.0,"1,5",69.0,44.0,26.0,30.0,30.2,1.00
4,1035.0,1.395.000,1.395e+06,1.395e+06,0,1.395.000,4.619,23 dagen,Vrijstaande woning,1751,...,36.0,24.0,40.0,"2,2",73.0,20.0,44.0,36.0,17.9,5.05
